In [ ]:
# Importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
## Setting certain variables for the program.
seed = 42
filterwarnings('ignore')

Custom Functions

In [ ]:
# Removing redundant features:
def redundant_features(master):
    redundants = []
    for i in master.columns:
        counts = master[i].value_counts()
        count_max = counts.iloc[0]
        if count_max / len(master) * 100 > 98: # if there is one value more than 98% in the entire dataset i.e. biased.
            redundants.append(i)
    redundants = list(redundants)
    return redundants

def drop_outliers(x):    
    for col in outl_col:
        Q1 = x[col].quantile(.25)
        Q3 = x[col].quantile(.99)
        IQR = Q3-Q1
        x =  x[(x[col] >= (Q1-(1.5*IQR))) & (x[col] <= (Q3+(1.5*IQR)))] 
    return x  

In [ ]:
master = pd.read_csv("train.csv")
master = master.drop(["Id"],axis=1)
print(master.shape)
master.head()

In [ ]:
## Checking Missing Data
master.isna().sum()

In [ ]:
master.describe()

## Data Cleaning

In [ ]:
# Checking the percentage of Null values in all the columns
print('Percentage of Missing Values in each column is as follows:')
print(round(master.isnull().sum()/len(master.index)*100,2).sort_values(ascending=False)[ round(master.isnull().sum()/len(master.index),2) > 0 ] )

<b>Obs</b>:</br>
5 features have large number missing values. Keeping a <b> arbitary threshold value of 80%</b>, the top four columns are removed.

In [ ]:
cols_drop = ["PoolQC","MiscFeature","Alley","Fence"]
master = master.drop(cols_drop,axis=1)
master.shape

In [ ]:
# Some columns have numerical values but have categorical meanings. Thus these needs to be ordered into categorical data type.
cols_cat = ["MSSubClass","OverallQual","OverallCond","MoSold"]

for col in cols_cat:
    master[col] = master[col].astype('category')

### Imputing the missing data with assumptions:

In [ ]:
master['FireplaceQu'] = master['FireplaceQu'].fillna('No_Fireplace')
master['GarageYrBlt'] = master['GarageYrBlt'].fillna(0)
master['MasVnrType'] = master['MasVnrType'].fillna('None')
master['MasVnrArea'] = master['MasVnrArea'].fillna(0)
master['MasVnrArea'] = master['MasVnrArea'].fillna(0)

#NA = No Garage (assumed)
master['GarageCond'] = master['GarageCond'].fillna('None')
master['GarageType'] = master['GarageType'].fillna('None')
master['GarageFinish'] = master['GarageFinish'].fillna('None')
master['GarageQual'] = master['GarageQual'].fillna('None')  

#NA = No Basement (assumed)
master['BsmtExposure'] = master['BsmtExposure'].fillna('None')
master['BsmtFinType2'] = master['BsmtFinType2'].fillna('None')
master['BsmtCond'] = master['BsmtCond'].fillna('None')
master['BsmtQual'] = master['BsmtQual'].fillna('None')
master['BsmtFinType1'] = master['BsmtFinType1'].fillna('None')

#LotFrontage : Replacing Null value with the median of the neighbourhood
master['LotFrontage'] = master.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Filling the Electrical rows with the mode
master['Electrical'] = master['Electrical'].fillna(master['Electrical'].mode()[0])

## Exploratory Data Analysis:

Categorical Features vs Sales Price

In [ ]:
# Plotting Categorical Features with Sale Price
def facetgrid_boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    x=plt.xticks(rotation=90)
    
categorical = master.select_dtypes(exclude=['int64','float64'])
f = pd.melt(master, id_vars=['SalePrice'], value_vars=sorted(master[categorical.columns]))
g = sns.FacetGrid(f, col="variable", col_wrap=3, sharex=False, sharey=False)
g = g.map(facetgrid_boxplot, "value", "SalePrice")

Some of the observations from above plot:
- Paved alleys properties has higher price.
- Houses where the basement quality is good and excellent are sold at higher prices.
- Houses with good and excellent garages are sold at higher prices.
- Houses with good quality kitchens has better prices.
- Houses with gas heating has good prices as well.

In [ ]:
# Creating a dataframe of numerical features:
master_num = master.select_dtypes(include=['int64','float64'])
master_num.columns

In [ ]:
#Visualising numerical predictor variables with Target Variables
fig,axs= plt.subplots(11,3,figsize=(20,80))
for i,ax in zip(master_num.columns,axs.flatten()):
    sns.scatterplot(x=i, y='SalePrice', hue='SalePrice',data=master_num,ax=ax,palette='vlag')
    plt.xlabel(i,fontsize=12)
    plt.ylabel('SalePrice',fontsize=12)
    ax.set_title('SalePrice'+' VS '+str(i))

In [ ]:
#Plotting heatmap of numerical features
plt.subplots(figsize = (25,20))
corr = master_num.corr()
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

sns.heatmap(round(corr,2), cmap='coolwarm' ,mask=mask, annot=True, center = 0)
plt.show()

## Data Preparation

In [ ]:
print(round(master.isnull().sum()/len(master.index)*100,2).sort_values(ascending=False)[round(master.isnull().sum()/len(master.index),2) > 0 ] )

## Feature Engineering

While doing EDA, it was noticed that certain categorical features represented over 95% of the dataset. This bias will not help in proper modelling. Thus it is optimal that these features be removed. <br>

While testing out the function it was noticed that 95% gave out 10 features whereas 98 % gave 6  features to remove. <br>

Thus it was arbitarily decided to use 98% as threshold. <br>

In [ ]:
redundant_features = redundant_features(master)
print(redundant_features)

In [ ]:
master = master.drop(redundant_features,axis=1)
master.shape

Handling Outliers

In [ ]:
# From EDA it was seen that Living Area, Garage Area, Basement Area and Lot Area. Removing outliers from these. Other outliers will be handled  during power transform.
outl_col = ['GrLivArea','GarageArea','TotalBsmtSF','LotArea'] 
master = drop_outliers(master)
master.shape

In [ ]:
#Creating some new features based on the existing features

master['YrBltAndRemod']=master['YearBuilt']+master['YearRemodAdd']

# Overall area:
master['Total_sqr_footage'] = (master['BsmtFinSF1'] + master['BsmtFinSF2'] + master['1stFlrSF'] + master['2ndFlrSF'])

# Total number of bathrooms:
master['Total_Bathrooms'] = (master['FullBath'] + (0.5 * master['HalfBath']) + master['BsmtFullBath'] + (0.5 * master['BsmtHalfBath']))

# Total porch area 
master['Total_porch_sf'] = (master['OpenPorchSF'] + master['EnclosedPorch'] + master['ScreenPorch'] + master['WoodDeckSF'])

In [ ]:
master.head()

---

## Data Modelling:

Creation of Dummy Variables

In [ ]:
#Creating Dummy Variables for Categorical Columns
num_col=[]
cat_col=[]

for i in master.columns:
    if master[i].dtypes in ["int64","float64"]:
        num_col.append(i)
    else:
        cat_col.append(i)
df = pd.get_dummies(master[cat_col],drop_first=True)
master=pd.concat([master,df],axis=1)
master= master.drop(cat_col,axis=1)        

In [ ]:
# Predictor Variables:
X = master.drop('SalePrice',axis=1)

# Target Variable
y = master['SalePrice']

In [ ]:
# Correlation coefficient threshold is arbitarily taken  as 0.7
threshold  = 0.7

# Checking co-related features
corr = X.corr()

corr1 = corr[abs(corr)>= threshold] 
corr2 =  corr.where(~np.tril(np.ones(corr.shape)).astype(np.bool))  #To remove repetition and 1 correlations

corr_result = corr2.stack()
print(corr_result[(abs(corr_result) > threshold)])

In [ ]:
col_drop = ["YearBuilt","YearRemodAdd","BsmtFinSF2","TotalBsmtSF","2ndFlrSF","GarageQual_TA","GarageCond_None","SaleType_New","SaleType_WD"]
X.drop(col_drop,axis=1,inplace=True)
X.shape

In [ ]:
#Train Test Split
size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size, random_state=seed)

In [ ]:
#Distribution of Target Variable
sns.distplot(y_train)
plt.show()

This datat is slightly right skewed. Thus a Power Transform to convert this to a Gaussian normal curve.

In [ ]:
#Transforming the Target feature to make the data gaussian
pt = PowerTransformer(method='box-cox', standardize=False)
y_train = pt.fit_transform(y_train.to_frame())
y_test = pt.transform(y_test.to_frame())

sns.distplot(y_train)
plt.show()

Feature Scaling: <br>

Using a MinMax scaler, the features will be scaled.

In [ ]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train)
X_train.columns = X.columns

X_test = scaler.transform(X_test)
X_test = pd.DataFrame(X_test)
X_test.columns = X.columns

In [ ]:
X_train.head()

## Ridge Regression (L2 Regularization)

In [ ]:
# list of alphas to tune
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}
ridge = Ridge()

# Using RFE to find top 300 variables
rfe = RFE(estimator=Ridge(), n_features_to_select=300)
rfe = rfe.fit(X_train,y_train)
col = X_train.columns[rfe.support_]
X_train_rfe = X_train[col]
X_test_rfe = X_test[col]

# cross validation
folds = 10
model_cv = GridSearchCV(estimator = ridge, param_grid = params, scoring= 'r2', cv = folds, return_train_score=True, verbose = 1)            
model_cv.fit(X_train_rfe, y_train) 

cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=30]
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('R2 Score')
plt.title("R2 Score and Alpha")
plt.legend(['train score', 'test score'], loc='upper right')
plt.xticks(np.arange(0,30,5))
plt.show()

alpha = cv_results['param_alpha'].loc[cv_results['mean_test_score'].idxmax()]
print('The optimum alpha is',alpha)
ridge_final = Ridge(alpha=alpha)
ridge_final.fit(X_train_rfe,y_train)
ridge_coef = ridge_final.coef_
y_test_pred = ridge_final.predict(X_test_rfe)
print('The R2 Score of the model on the test dataset for optimum alpha is',r2_score(y_test, y_test_pred))
print('The MSE of the model on the test dataset for optimum alpha is', mean_squared_error(y_test, y_test_pred))

In [ ]:
## USing VIF to remove  features:
vif = pd.DataFrame()
vif['Features'] = X_train_rfe.columns
vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
high_vif = vif[vif['VIF']>10]
high_vif

In [ ]:
# Even though Total Square Footage has a high VIF, it is one of the most important features to predict the cost.
tot_sq_index = high_vif.loc[high_vif.Features == "Total_sqr_footage"].index.tolist()
high_vif.drop(tot_sq_index,inplace=True)

In [ ]:
# Dropping cols with high VIF value.
X_train_rfe2 = X_train_rfe.drop(high_vif.Features,axis=1)
X_test_rfe2 = X_test_rfe.drop(high_vif.Features,axis=1)

# This gives columns without Multicollinearity.

In [ ]:
# Building the second Ridge Model
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100, 500, 1000 ]}
ridge = Ridge(random_state=100)

# cross validation
folds = 10
model_cv = GridSearchCV(estimator = ridge, param_grid = params, scoring= 'r2', cv = folds, return_train_score=True, verbose = 1)            
model_cv.fit(X_train_rfe2, y_train) 

cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=30]
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('R2 Score')
plt.title("R2 Score and Alpha")
plt.legend(['train score', 'test score'], loc='upper right')
plt.xticks(np.arange(0,30,5))
plt.show()

alpha = cv_results['param_alpha'].loc[cv_results['mean_test_score'].idxmax()]
print('The optimum alpha is',alpha)
ridge_final2 = Ridge(alpha=alpha,random_state=100)
ridge_final2.fit(X_train_rfe2,y_train)
ridge_coef2 = ridge_final2.coef_
y_test_pred = ridge_final2.predict(X_test_rfe2)
print('The R2 Score of the model on the test dataset for optimum alpha is',r2_score(y_test, y_test_pred))
print('The MSE of the model on the test dataset for optimum alpha is', mean_squared_error(y_test, y_test_pred))

In [ ]:
#Coefficients of the Model:
ridge_coeff2 = pd.DataFrame(np.atleast_2d(ridge_coef2),columns=X_train_rfe2.columns)
ridge_coeff2 = ridge_coeff2.T
ridge_coeff2.rename(columns={0: 'Ridge Co-Efficient'},inplace=True)
ridge_coeff2.sort_values(by=['Ridge Co-Efficient'], ascending=False,inplace=True)
ridge_coeff2.head(20)

The above features are the top 20 features likely impacting the property value.

---

## Lasso Regression (L1 Regularization)

In [ ]:
# Creating a model with an arbitrary alpha to understand the value ranges
lasso1 = Lasso(alpha=0.0001)        
lasso1.fit(X_train_rfe2, y_train) 

y_test_pred = lasso1.predict(X_test_rfe2)
print('The R2 Score of the model on the test dataset for 0.0001 alpha is',r2_score(y_test, y_test_pred))
print('The MSE of the model on the test dataset for optimum alpha is', mean_squared_error(y_test, y_test_pred))

In [ ]:
# Lasso Model with GridSearch CV to find the optimum alpha.

params = {'alpha': [0.00001, 0.00009, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009 ]}
lasso = Lasso(random_state=100)

# cross validation
folds = 10
model_cv = GridSearchCV(estimator = lasso, param_grid = params, scoring= 'r2', cv = folds, return_train_score=True, verbose = 1)            
model_cv.fit(X_train_rfe2, y_train) 

cv_results = pd.DataFrame(model_cv.cv_results_)
# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('R2 Score')
plt.title("R2 Score and Alpha")
plt.legend(['train score', 'test score'], loc='upper right')
plt.show()

alpha = cv_results['param_alpha'].loc[cv_results['mean_test_score'].idxmax()]
print('The optimum alpha is',alpha)
lasso_final2 = Lasso(alpha=alpha,random_state=100)
lasso_final2.fit(X_train_rfe2,y_train)
lasso_coef2 = lasso_final2.coef_
y_test_pred = lasso_final2.predict(X_test_rfe2)
print('The R2 Score of the model on the test dataset for optimum alpha is',r2_score(y_test, y_test_pred))
print('The MSE of the model on the test dataset for optimum alpha is', mean_squared_error(y_test, y_test_pred))

In [ ]:
#Co-efficients of the model
lasso_coeff2 = pd.DataFrame(np.atleast_2d(lasso_coef2),columns=X_train_rfe2.columns)
lasso_coeff2 = lasso_coeff2.T
lasso_coeff2.rename(columns={0: "Lasso Co-Efficient"},inplace=True)
lasso_coeff2.sort_values(by=['Lasso Co-Efficient'], ascending=False,inplace=True)
lasso_coeff2.head(20)

The above are the top 20 features from the Lasso model that can be used to predict the price.

---

Final Ridge regression Model:

In [ ]:
ridge_final2

Final Lasso Regression Model

In [ ]:
lasso_final2

----

## Subjective Questions Demonstration

Q1. <br>
What is the optimal value of alpha for ridge and lasso regression? <br>
What will be the changes in the model if you choose double the value of alpha for both ridge and lasso? <br>
What will be the most important predictor variables after the change is implemented?<br>

The optimal value of alpha for Ridge regression is <b>3</b> and Lasso is <b>0.0006</b>

In [ ]:
# Building Ridge Model by doubling the value of alpha to 6
ridge_double = Ridge(alpha=6,random_state=seed)
ridge_double.fit(X_train_rfe2,y_train)
ridge_double_coef = ridge_double.coef_
y_test_pred = ridge_double.predict(X_test_rfe2)

print('The R2 Score of the model on the test dataset for doubled alpha is',r2_score(y_test, y_test_pred))
print('The MSE of the model on the test dataset for doubled alpha is', mean_squared_error(y_test, y_test_pred))

ridge_double_coeff = pd.DataFrame(np.atleast_2d(ridge_double_coef),columns=X_train_rfe2.columns)
ridge_double_coeff = ridge_double_coeff.T
ridge_double_coeff.rename(columns={0: 'Ridge Doubled Alpha Co-Efficient'},inplace=True)
ridge_double_coeff.sort_values(by=['Ridge Doubled Alpha Co-Efficient'], ascending=False,inplace=True)

print('\n The most important predictor variables are as follows:')
ridge_double_coeff.head(20)

In [ ]:
# Building Lasso Model by doubling the value of alpha to 0.0006
lasso_double = Lasso(alpha=0.0006,random_state=seed)
lasso_double.fit(X_train_rfe2,y_train)
lasso_double_coef = lasso_double.coef_
y_test_pred = lasso_double.predict(X_test_rfe2)
print('The R2 Score of the model on the test dataset for doubled alpha is',r2_score(y_test, y_test_pred))
print('The MSE of the model on the test dataset for doubled alpha is', mean_squared_error(y_test, y_test_pred))

lasso_double_coeff = pd.DataFrame(np.atleast_2d(lasso_double_coef),columns=X_train_rfe2.columns)
lasso_double_coeff = lasso_double_coeff.T
lasso_double_coeff.rename(columns={0: 'Lasso Doubled Alpha Co-Efficient'},inplace=True)
lasso_double_coeff.sort_values(by=['Lasso Doubled Alpha Co-Efficient'], ascending=False,inplace=True)

print('\n The most important predictor variables are as follows:')
lasso_double_coeff.head(20)

Q3. <br>
After building the model, you realised that the five most important predictor variables in the lasso model are not available in the incoming data.<br>You will now have to create another model excluding the five most important predictor variables. Which are the five most important predictor variables now?

In [ ]:
#Removing the 5 most important predictor variables from the incoming dataset
col_drop = ["Total_sqr_footage","GarageCars","TotRmsAbvGrd","Fireplaces","GarageArea"]

X_test_rfe3 = X_test_rfe2.drop(col_drop,axis=1)
X_train_rfe3 = X_train_rfe2.drop(col_drop,axis=1)

# Building Lasso Model with the new dataset
lasso3 = Lasso(alpha=0.0001,random_state=100)
lasso3.fit(X_train_rfe3,y_train)
lasso3_coef = lasso3.coef_
y_test_pred = lasso3.predict(X_test_rfe3)

print('The R2 Score of the model on the test dataset is',r2_score(y_test, y_test_pred))
print('The MSE of the model on the test dataset is', mean_squared_error(y_test, y_test_pred))

lasso3_coeff = pd.DataFrame(np.atleast_2d(lasso3_coef),columns=X_train_rfe3.columns)
lasso3_coeff = lasso3_coeff.T
lasso3_coeff.rename(columns={0: 'Lasso Co-Efficient'},inplace=True)
lasso3_coeff.sort_values(by=['Lasso Co-Efficient'], ascending=False,inplace=True)

print('\n The most important predictor variables are as follows:')
lasso3_coeff.head(5)

------